In [1]:
import numpy as np
import pandas as pd
import cPickle as pickle
from datetime import datetime
from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

## Bids

In [2]:
bids = pd.read_csv("../../Data/lending/bid_notick.txt", sep="|")
# Filter winning bids
bids = bids[bids.Status.isin(["Winning", "Partially Participating"])]
# Remove current listings
bids = bids[bids.ListingStatus.isin(["Expired", "Completed"])]
# Filter relevant columns
bids = bids[["ListingKey", "Bid_Key", "MemberKey", "CreationDate", "Amount", "ListingStatus"]]
# Convert creation date to datetime format
bids["CreationDate"] = pd.to_datetime(bids.CreationDate)
# Variable /YYYY/MM to allow monthly time unit analysis
bids["time_trunc"] = bids.CreationDate.map(lambda x: x.strftime('%Y%m'))
# Rename status colunmn for consistency with other projects
bids = bids.rename(index=str, columns={"ListingStatus": "Status"})
# Exclude these bids in preliminary analysis to correspond to final listing list
with open("../../Data/lending/final_listings", 'rb') as f:
    filtered = pickle.load(f)
bids = bids[bids['ListingKey'].isin(filtered)] 
# Show sample bids
bids.sample(5)

,ListingKey,Bid_Key,MemberKey,CreationDate,Amount,Status,time_trunc
2032267,84B133945199612709DB08D,66313497642617281F30330,3B043366342183806EA5D33,2007-07-10 15:51:25,50.000,Completed,200707
3412818,8E0A3409756585025F66BED,5A523513202868407D55DA0,541534047010026075EBA11,2008-01-12 21:54:54,50.000,Expired,200801
6462053,1D79341881940842914C9B7,220235882701504440FE5B2,A38433711100206687D3E99,2010-05-10 18:26:43,25.000,Completed,201005
4198432,1B2C34180097436185198A7,F909352137028252955E1E5,F03533861525517625E0E52,2008-04-10 13:11:54,50.000,Expired,200804
4928957,BDA734242072609714B4CB5,A0A135262976934818F7189,A2F63406970196509388CCD,2008-06-10 11:17:25,50.000,Expired,200806


## Listings

In [3]:
listings = pd.read_csv('../../Data/lending/listing.txt', sep="|")
# Select relevant columns
columns = ["Lst_Key", "AmountRequested", "AmountFunded", "StartDate", "EndDate", "Status", "BorrowerRate", "ProsperScore", "Duration", "CreditGrade", "DebtToIncomeRatio", "IsBorrowerHomeowner"]
listings = listings[columns]
# Rename columns
listings = listings.rename(index=str, columns={"Lst_Key": "ProjectID",
                                               "AmountFunded": "AmountRaised"})
# Pre-process credit information
creditmap = {'AA':8, 'A':7, 'B':6, 'C':5, 'D':4, 'E':3, 'HR':2, 'NC':1}
listings['CreditGrade'] = listings.CreditGrade.fillna(9)
listings = listings.applymap(lambda s: creditmap.get(s) if s in creditmap else s)
# Drop records with no credit information
listings = listings[listings['CreditGrade']<9] 
# Remove current listings
listings = listings[listings.Status.isin(["Expired", "Completed"])]
# Remove listings that raised $0
listings = listings[listings.AmountRaised > 0]
# Show sample listings
listings.head(5)

,ProjectID,AmountRequested,AmountRaised,StartDate,EndDate,Status,BorrowerRate,ProsperScore,Duration,CreditGrade,DebtToIncomeRatio,IsBorrowerHomeowner
56,68463366021964690748937,2995.000,1050.680,2006-06-22 14:50:57,2006-07-02 14:51:13,Expired,0.287,nan,10,2,0.040,False
57,68C933650780012599F665D,1500.000,200.000,2006-06-23 12:48:26,2006-06-30 12:50:36,Expired,0.237,nan,7,2,0.150,False
72,32FB33655503533550522A0,4000.000,650.000,2006-07-22 15:40:03,2006-07-29 15:41:06,Expired,0.070,nan,7,8,0.060,True
74,AF5833649932087691B64A6,2000.000,825.000,2006-07-28 10:04:50,2006-08-04 10:07:57,Expired,0.290,nan,7,2,0.190,True
80,5D553365615243726AB58E0,5000.000,300.000,2006-08-07 14:29:47,2006-08-14 14:33:56,Expired,0.290,nan,7,2,0.120,True


## Listing Keys & Amount Requested Map

In [4]:
# Create listing Key Variable
listing_keys = list(set(bids['ListingKey']))
assert len(listing_keys) == len(listings)

# Map ProjectID to AmountRequested
amount_requested_map = dict(zip(listings.ProjectID, listings.AmountRequested))

## Map Listing Keys to Bid DFs

In [5]:
# Map Listing Keys to Bids Dataframes
listing_bids_map = {}
for listing_key in tqdm_notebook(listing_keys):
    var_df = bids.query("ListingKey == '{0}'".format(listing_key)).sort_values(by='CreationDate')
    # Create bid amounts series 
    series = var_df['Amount'].fillna(0)
    # Convert series to CDF
    norm_cdf = np.cumsum(series)
    var_df['cdf'] = norm_cdf
    listing_bids_map[listing_key] = var_df

## Bids Filtering Method

In [6]:
# Take bids that contribute t% of the total amount raised
def filter_bids_by_amount(threshold):
    bids_ = [] # Global bids dataframes list
    threshold = threshold # Cut-off point
    # Iterate through all listing keys
    for listing_key in tqdm_notebook(listing_keys):
        # Get bids dataframe for each listing key
        # Bids in dataframe are sorted chronologically
        var_df = listing_bids_map[listing_key]
        # Limit based on threshold
        limit = amount_requested_map.get(listing_key)*threshold
        # If project raised more than threshold
        if (len(var_df)>0) and (max(var_df['cdf']) >= limit):
            # Get bids that comprise <= threshold
            var_df = var_df[var_df['cdf']<=limit] 
            # add filtered bids dataframe to temporary list        
            bids_.append(var_df)
    return pd.concat(bids_, axis=0)

## Filter Bids

In [7]:
bids_10 = filter_bids_by_amount(0.10)
print bids_10['ListingKey'].nunique() 
bids_10.to_csv("../../Data/lending/filtered/bids_amount_10.csv", index=False)


53810


In [8]:
bids_20 = filter_bids_by_amount(0.20)
print bids_20['ListingKey'].nunique() 
bids_20.to_csv("../../Data/lending/filtered/bids_amount_20.csv", index=False)


42363


In [9]:
# bids_25 = filter_bids_by_amount(0.25)
# print bids_25['ListingKey'].nunique() 
# bids_25.to_csv("../../Data/lending/filtered/bids_amount_25.csv", index=False)

In [10]:
bids_30 = filter_bids_by_amount(0.30)
print bids_30['ListingKey'].nunique() 
bids_30.to_csv("../../Data/lending/filtered/bids_amount_30.csv", index=False)


37484


In [11]:
bids_40 = filter_bids_by_amount(0.40)
print bids_40['ListingKey'].nunique() 
bids_40.to_csv("../../Data/lending/filtered/bids_amount_40.csv", index=False)


34340


In [12]:
bids_50 = filter_bids_by_amount(0.50)
print bids_50['ListingKey'].nunique() 
bids_50.to_csv("../../Data/lending/filtered/bids_amount_50.csv", index=False)


32409


In [13]:
bids_60 = filter_bids_by_amount(0.60)
print bids_60['ListingKey'].nunique() 
bids_60.to_csv("../../Data/lending/filtered/bids_amount_60.csv", index=False)


31104


In [14]:
bids_70 = filter_bids_by_amount(0.70)
print bids_70['ListingKey'].nunique() 
bids_70.to_csv("../../Data/lending/filtered/bids_amount_70.csv", index=False)


30065


In [15]:
# bids_75 = filter_bids_by_amount(0.75)
# print bids_75['ListingKey'].nunique() 
# bids_75.to_csv("../../Data/lending/filtered/bids_amount_75.csv", index=False)

In [16]:
bids_80 = filter_bids_by_amount(0.80)
print bids_80['ListingKey'].nunique() 
bids_80.to_csv("../../Data/lending/filtered/bids_amount_80.csv", index=False)


29401


In [17]:
bids_90 = filter_bids_by_amount(0.90)
print bids_90['ListingKey'].nunique() 
bids_90.to_csv("../../Data/lending/filtered/bids_amount_90.csv", index=False)


28934


In [18]:
bids_100 = filter_bids_by_amount(1.00)
print bids_100['ListingKey'].nunique() 
bids_100.to_csv("../../Data/lending/filtered/bids_amount_100.csv", index=False)


28913
